In [1]:
import os

In [2]:
print(os.environ['JAVA_HOME'])

/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home


In [38]:
print(os.environ['SPARK_HOME'])

/opt/spark


In [4]:
import findspark
findspark.init()

In [5]:
import pyspark
findspark.find()

'/opt/spark'

In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## 컨텍스트 환경 설정하기 

   - master : 로컬이나 분산에 대한 정보
    - addname : 어플리케이션 이름 

## 매개변수 설명

class pyspark.SparkContext (
   master = None,                      ## 연결된 클러스터의 URL입니다.
   
   appName = None,                  ## 작업 이름.
   
   sparkHome = None,               ## 스파크 설치 디렉토리.
   
   pyFiles = None,                      ## 클러스터로 보내고 PYTHONPATH에 추가 할 .zip 또는 .py 파일.
   
   environment = None,             ## 작업자 노드 환경 변수.
   
   batchSize = 0,                        ## 단일 Java 객체로 표시되는 Python 객체의 수입니다. 배치를 비활성화하려면 1을 설정하고, 
                                                  ## 객체 크기에 따라 배치 크기를 자동으로 선택하려면 0을 설정하고, 무제한 배치 크기를 사용하려면 -1을 설정하십시오.
                                                  
   serializer = PickleSerializer(),  ## RDD 시리얼 라이저
   
   conf = None,                           ## L의 목적은 {SparkConf}는 모든 스파크 속성을 설정합니다.
   
   gateway = None,                     ## 기존 게이트웨이와 JVM을 사용하고 그렇지 않으면 새 JVM을 초기화합니다.
   
   jsc = None,                               ## JavaSparkContext 인스턴스
   
   profiler_cls = <class 'pyspark.profiler.BasicProfiler'>   ## 프로파일 링에 사용되는 사용자 정의 프로파일 러 클래스 (기본값은 pyspark.profiler.BasicProfiler)
)

## 환경에 세팅 

### 컨피그 세팅 메소드 

- set (key, value) − 구성 속성을 설정합니다.

- setMaster (value) -마스터 URL을 설정합니다.

- setAppName (value) -응용 프로그램 이름을 설정합니다.

- get (key, defaultValue = None) − 키의 구성 값을 가져옵니다.

- setSparkHome (value) − 작업자 노드에서 Spark 설치 경로를 설정합니다.

In [7]:
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')

## 스파크 컨텍스트 만들기

### 직접 세팅 

   - 직접 스파크 컨텍스트를 사용해서 환경을 지정한다

      sc = SparkContext("local", "Accumulator app") 

### 컨피그 사용하기

   -  컨피그레이션에 세팅된 정보를 활용해서 스파크컨텍스트를 만든다

In [8]:
sc = pyspark.SparkContext(conf=conf)

In [9]:
sc

<SparkContext master=local appName=appName>

## 스파크 세션 만들기

In [25]:
 SparkSession

pyspark.sql.session.SparkSession

In [10]:
spark = SparkSession(sc)

In [11]:
spark

## RDD 생성하기 

In [12]:
data = sc.parallelize([('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)])


In [13]:
data

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [20]:
data.take(1)

[('Amber', 22)]

In [14]:
%ls

1910.13233.pdf
Learning-PySpark.pdf
RDD 생성하기.ipynb
README.md
Untitled.ipynb
data/
pyspark 기본처리_1.ipynb
pyspark 기본처리_1111.ipynb
pyspark.pdf
pyspark_2.pdf
파이스파크  알아보기.ipynb
파이스파크_시작과 종료하는방법.ipynb


## 파일로 RDD 생성하기

In [17]:
data_from_file = sc.textFile('./data/titanic.csv')


In [18]:
data_from_file.take(1)

['Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare']

In [19]:
data_from_file.take(2)

['Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare',
 '0,3,Mr. Owen Harris Braund,male,22,1,0,7.25']

In [21]:
words = sc.parallelize (
   ["scala", 
   "java", 
   "hadoop", 
   "spark", 
   "akka",
   "spark vs hadoop", 
   "pyspark",
   "pyspark and spark"]
)

In [22]:
words

ParallelCollectionRDD[8] at parallelize at PythonRDD.scala:195

### RDD 객체 내의 원소 수

In [23]:
words.count()

8

### RDD  객체의 모든 원소 검색

In [26]:
words.collect()

['scala',
 'java',
 'hadoop',
 'spark',
 'akka',
 'spark vs hadoop',
 'pyspark',
 'pyspark and spark']

In [50]:
words.cache() 
caching = words.persist().is_cached 

In [51]:
caching

True

### reduce 내부의 함수 조건을 만족하는 요소 만 반환합니다

In [43]:
def add(x,y) :
    return x+y

In [44]:
nums = sc.parallelize([1, 2, 3, 4, 5])
adding = nums.reduce(add)

In [45]:
adding

15

### 두 개의 RDD를 합치기

In [46]:
x = sc.parallelize([("spark", 1), ("hadoop", 4)])
y = sc.parallelize([("spark", 2), ("hadoop", 5)])

In [48]:
joined = x.join(y)

In [49]:
joined.collect()

[('hadoop', (4, 5)), ('spark', (1, 2))]

## 누산기 사용하기

In [52]:
num = sc.accumulator(10) 
def f(x): 
    global num 
    num+=x 


In [53]:
rdd = sc.parallelize([20,30,40,50]) 
rdd.foreach(f) 

In [54]:
final = num.value 

In [55]:
final

150